In [1]:
import pandas as pd

In [2]:
## Read in Clients from nettxt file
def client_finder(file):
    for line in file:
        if 'Client' in line:
             yield line

source = open('data/Kismet-20170402-20-33-58-1-Copy1.txt','r')

clients = client_finder(source)

In [3]:
df = pd.DataFrame(columns=['client', 'MAC'])

In [4]:
## Add Clients to dataframe
for x in clients:
    df = df.append(pd.Series(x.split(':',1), index=['client','MAC']), ignore_index=True)

In [5]:
df.head(10)

,client,MAC
0,Client 1,MAC 00:00:00:00:00:00\n
1,Client 1,MAC 00:0E:58:E0:9D:99\n
2,Client 1,MAC 00:13:EF:80:14:7F\n
3,Client 1,MAC 00:1A:4D:3E:25:04\n
4,Client 1,MAC 00:1D:7D:4B:61:35\n
5,Client 1,MAC 00:30:44:15:FA:5D\n
6,Client 1,MAC 00:34:DA:12:3E:46\n
7,Client 1,MAC 00:34:DA:92:A0:19\n
8,Client 1,MAC 00:87:01:24:06:D9\n
9,Client 1,MAC 00:A0:0A:AA:6A:3D\n


In [6]:
## Clean up MACA Address column
df.MAC = df.MAC.str.replace('MAC ' , '')
df.MAC = df.MAC.str.replace(r'\n','')

In [7]:
df.MAC.head()

0     00:00:00:00:00:00
1     00:0E:58:E0:9D:99
2     00:13:EF:80:14:7F
3     00:1A:4D:3E:25:04
4     00:1D:7D:4B:61:35
Name: MAC, dtype: object

In [10]:
## Get the number of Unique MAC addresses
num_macs = df.MAC.unique()
len(num_macs)

327

In [11]:
## MAC addresses and the number of times they appear in the log
df.MAC.value_counts()

 B6:25:71:3B:2E:8C    8
 BC:16:65:D0:04:00    4
 A4:08:EA:8B:12:22    3
 8C:79:67:EC:DA:8B    3
 BC:16:65:F4:73:00    3
 60:E3:AC:E4:8B:24    3
 E8:50:8B:4D:63:F5    2
 EC:9B:F3:C6:30:C5    2
 B0:70:2D:C0:C0:3B    2
 8C:F5:A3:31:12:20    2
 2C:0E:3D:99:62:9B    2
 AC:5F:3E:63:D2:CA    2
 DC:CF:96:C3:0A:40    2
 E8:50:8B:45:3E:ED    2
 58:BF:EA:BF:1E:40    2
 50:9E:A7:76:93:FD    2
 8C:F5:A3:14:A9:5A    2
 DC:0B:34:9F:FC:58    2
 2C:0E:3D:97:59:3A    2
 A4:4E:31:1C:50:C4    1
 CC:D5:39:27:EB:C2    1
 06:13:29:47:98:E1    1
 8E:4C:12:6E:B3:B9    1
 DC:CF:96:97:CA:29    1
 E8:91:20:41:27:5A    1
 AC:BC:32:C3:92:5B    1
 CC:D5:39:27:EB:C3    1
 84:89:AD:B5:C9:C0    1
 E4:F4:C6:05:B6:E1    1
 88:66:A5:B9:71:06    1
                     ..
 30:CD:A7:A8:C6:33    1
 6C:AA:B3:28:34:78    1
 A0:99:9B:0A:1C:87    1
 24:C9:A1:CE:44:A8    1
 DA:A1:19:09:D5:D8    1
 B8:27:EB:68:25:5E    1
 F0:24:75:B2:B9:2B    1
 10:05:CA:53:00:10    1
 84:11:9E:7E:D9:D6    1
 00:AE:FA:47:B4:38    1
 2C:59:8A:01:53: